# COVID-19 Web Data Scraper

### Python Script for scraping & storing data in Table (csv file format)

In [1]:
import pandas as pd
#supress warnings
import warnings
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup

USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
# US english
LANGUAGE = "en-US,en;q=0.5"



def get_all_tables(soup):
    """Extracts and returns all tables in a soup object"""
    return soup.find_all("table")

def get_table_headers(table):
    """Given a table soup, returns all the headers"""
    headers = []
    for th in table.find("tr").find_all("th"):
        headers.append(th.text.strip())
    return headers


def get_table_rows(table):
    """Given a table, returns all its rows"""
    rows = []
    for tr in table.find_all("tr")[1:]:
        cells = []
        # grab all td tags in this table row
        tds = tr.find_all("td")
        if len(tds) == 0:
            # if no td tags, search for th tags
            # can be found especially in wikipedia tables below the table
            ths = tr.find_all("th")
            for th in ths:
                cells.append(th.text.strip())
        else:
            # use regular td tags
            for td in tds:
                cells.append(td.text.strip())
        rows.append(cells)
    return rows

def save_as_csv(table_name, headers, rows):
    pd.DataFrame(rows, columns=headers).to_csv(f"{table_name}.csv")
    
    
def main(URL):    
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, 'html.parser')
    # extract all the tables from the web page
    tables = get_all_tables(soup)
    print(f"[+] Found a total of {len(tables)} tables.")
    # iterate over all tables
    for i, table in enumerate(tables, start=1):
        # get the table headers
        headers = get_table_headers(table)
        # get all the rows of the table
        rows = get_table_rows(table)
        # save table as csv file
        table_name = f"table-{i}"
        if i==1:
            print(f"[+] Saving {table_name}")
            save_as_csv(table_name, headers, rows)
            break
        
if __name__ == "__main__":
    import sys
    try:
        URL = 'https://www.worldometers.info/coronavirus/'
    except IndexError:
        exit(1)
    main(URL)

[+] Found a total of 3 tables.
[+] Saving table-1


In [2]:
df = pd.read_csv('table-1.csv')
df = df[8:223]
df.head()

,Unnamed: 0,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
8,8,1.0,USA,"5,415,666",NaN,"170,415",NaN,"2,843,204",NaN,"2,402,047","17,239","16,350",514,"68,524,602","206,876","331,235,184",North America,61,"1,944",5
9,9,2.0,Brazil,"3,229,621",NaN,"105,564",NaN,"2,356,640",NaN,"767,417","8,318","15,181",496,"13,464,336","63,290","212,739,546",South America,66,"2,015",16
10,10,3.0,India,"2,459,613",NaN,"48,144",NaN,"1,750,636",NaN,"660,833","8,944","1,780",35,"27,694,416","20,045","1,381,604,279",Asia,562,"28,697",50
11,11,4.0,Russia,"907,758",NaN,"15,384",NaN,"716,396",NaN,"175,978","2,300","6,220",105,"31,598,302","216,513","145,942,116",Europe,161,"9,487",5
12,12,5.0,South Africa,"572,865",NaN,"11,270",NaN,"437,617",NaN,"123,978",539,"9,645",190,"3,315,497","55,820","59,395,867",Africa,104,"5,270",18
